In [1]:
import numpy as np
from tqdm import tqdm
import xarray as xr
import matplotlib.pyplot as plt
import warnings
from itertools import product
# from lmphoton.simulation import current_simulation as sim
from scipy.optimize import minimize
from scipy.constants import constants
import pandas as pd
import seaborn as sns

import sys
sys.path.append("../")

# SPPD optical elements
from src.sppd_dfb_laser import SPPDDFB, SPPDCLMPIC
from src.enablence_mux import EBMUX
from src.sppd_clm_optics import SPPDCLMOPTICS, SPPDCLMOPTICS_LAMBDA_AB
# PMIC drivers
from src.pikeriver_pmic import ADC, DAC, TIA, PMIC
# Calibration and control
from sppd_clm_calibration import SPPDCLMCAL2
from src.sppd_clm import CONTROLLER

plt.style.use("plot_style.mplstyle")
warnings.filterwarnings("ignore")

In [2]:
enablence_mux_bw_lambda = np.array([20])/17*0.1e-9
enablence_mux_3sigma = np.array([0.1])/17*0.1e-9
sppd_laser_3sigma = np.array([38])/17*0.1e-9
no_mux = np.array([0])
wpe_slope_3sigma = np.array([0.01, 0.15, 0.3])

no_lasers = 16
no_iterations = 5120

In [3]:
enablence_mux_mc = np.zeros((no_iterations, no_lasers, len(enablence_mux_3sigma)))
sppd_laser_mc = np.zeros((no_iterations, no_lasers, len(sppd_laser_3sigma)))
wpe_slope_factor_mc = np.zeros((no_iterations, no_lasers, len(wpe_slope_3sigma)))

for idd_iter in range(no_iterations):
  for idd_eb in range(len(enablence_mux_3sigma)):
      enablence_mux_mc[idd_iter, :, idd_eb] = np.random.normal(0, enablence_mux_3sigma[idd_eb]/3) * np.ones(no_lasers)
  for idd_laser in range(no_lasers):
    for idd_sppd in range(len(sppd_laser_3sigma)):
      sppd_laser_mc[idd_iter, idd_laser, idd_sppd] = np.random.normal(0, sppd_laser_3sigma[idd_sppd]/3)
    for idd_wpe in range(len(wpe_slope_3sigma)):
      wpe_slope_factor_mc[idd_iter, idd_laser, idd_wpe] = np.random.normal(1.0, wpe_slope_3sigma[idd_wpe]/3)

In [4]:
target_grid_array = np.array([
  1301.47, 1302.60, 1303.73, 1304.87,
  1306.01, 1307.14, 1308.28, 1309.43,
  1310.57, 1311.72, 1312.87, 1314.02,
  1315.17, 1316.33, 1317.48, 1318.64
  ])*1e-9 

wavelength_fabrication = np.array([
  1301.47, 1302.60, 1303.73, 1304.87,
  1306.01, 1307.14, 1308.28, 1309.43,
  1310.57, 1311.72, 1312.87, 1314.02,
  1315.17, 1316.33, 1317.48, 1318.64
  ])*1e-9

In [5]:
nom_temp_list, tec_set_point_temperature_list, mux_current_list, peak_current_list, output_power_list, output_lambda_list, current_list, cal_temperature_list, drive_current_list, warning_flag_list, junction_temperature_list = [], [], [], [], [], [], [], [], [], [], []

for idd_mux, idd_iter, idd_eb, idd_sppd, idd_bw_lambda, idd_wpe in tqdm(product(range(len(no_mux)), range(no_iterations), range(len(enablence_mux_3sigma)), range(len(sppd_laser_3sigma)), range(len(enablence_mux_bw_lambda)), range(len(wpe_slope_3sigma))), total=no_iterations*len(enablence_mux_bw_lambda)*len(sppd_laser_3sigma)*len(enablence_mux_3sigma)*len(no_mux)*len(wpe_slope_3sigma)):
  enablence_grid = target_grid_array + enablence_mux_mc[idd_iter, :, idd_eb]
  sppd_grid = target_grid_array + sppd_laser_mc[idd_iter, :, idd_sppd] - 0.5e-9
  wpe_slope_factor = wpe_slope_factor_mc[idd_iter, :, idd_wpe]

  # Create the SPPD DFB laser
  _sppd_clm_optics_mc = SPPDCLMOPTICS_LAMBDA_AB(
    current = 0.133,
    wavelength_fabrication=sppd_grid,
    target_grid_array=enablence_grid,
    bw_lambda = enablence_mux_bw_lambda[idd_bw_lambda],
    nominal_junction_temperature=273+44.5,
    wpe_slope_factor = wpe_slope_factor,
    name = "SPPDCLMOPTICS_AB_2",
    no_mux = idd_mux + 1
  )

  _sppd_clm_optics_mc.update(tc = 273+42.5, i_array=np.ones(16) * 0.133)

  _pk_pmic = PMIC()

  controller = CONTROLLER(
    sppd_clm_optics=_sppd_clm_optics_mc,
    pmic = _pk_pmic,
    drive_current_array = np.ones(16) * 0.133,
  )

  controller.calibrate(tc=273+35,
    temperature_array = np.linspace(30, 50, 51)+273, 
    drive_current_array= np.linspace(0.1, 0.2, 41),
    power_wavelength_calibration=True,
    tec_temperature_tuning=True,
    drive_current_tuning=True,
    percent_range = np.linspace(0.5, 1.5, 41)
  )

  nom_temp_list += controller._nominal_temperature_array.tolist()
  tec_set_point_temperature_list += [controller._tec_set_point_temperature]
  warning_flag_list += [controller._warning_flag]
  peak_current_list += controller._pmic_adc_enablence_mux_mpd_peak_current_array.tolist()
  output_power_list += controller._enablence_output_power_array.tolist()
  output_lambda_list += controller._lambda_array.tolist()
  drive_current_list += controller._pmic._dac_laser_drive_current_array.tolist()
  junction_temperature_list += controller._laser_junction_temperature_array.tolist()
  

100%|██████████| 15360/15360 [22:25:57<00:00,  5.26s/it]  


In [6]:
nom_temp_array = np.array(nom_temp_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), no_lasers)
tec_set_point_temperature = np.array(tec_set_point_temperature_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma))
warning_flag = np.array(warning_flag_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma))
peak_current_array = np.array(peak_current_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), no_lasers)
output_power_array = np.array(output_power_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), no_lasers)
output_lambda_array = np.array(output_lambda_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), no_lasers)
drive_current_array = np.array(drive_current_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), no_lasers)
junction_temperature_array = np.array(drive_current_list).reshape(len(no_mux), no_iterations, len(enablence_mux_3sigma), len(sppd_laser_3sigma), len(enablence_mux_bw_lambda), len(wpe_slope_3sigma), no_lasers)
data = xr.Dataset(
  data_vars = dict(
    nom_temp_array = (["idd_mux", "idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe",  "idd_laser"], nom_temp_array),
    tec_set_point_temperature = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda","idd_wpe"], tec_set_point_temperature),
    warning_flag = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe"], warning_flag),
    peak_current_array = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe", "idd_laser"], peak_current_array),
    output_power_array = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe", "idd_laser"], output_power_array),
    output_lambda_array = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe", "idd_laser"], output_lambda_array),
    drive_current_array = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe", "idd_laser"], drive_current_array),
    enablence_mux_mc = (["idd_iter", "idd_laser", "idd_eb"], enablence_mux_mc),
    sppd_laser_mc = (["idd_iter", "idd_laser", "idd_sppd"], sppd_laser_mc),
    wpe_slope_factor_mc = (["idd_iter", "idd_laser", "idd_wpe"], wpe_slope_factor_mc),
    junction_temperature_mc = (["idd_mux","idd_iter", "idd_eb", "idd_sppd", "idd_bw_lambda", "idd_wpe", "idd_laser"], junction_temperature_array)
   ),
  coords=dict(
    idd_mux = np.array([0]),
    idd_iter = range(no_iterations),
    idd_eb = range(len(enablence_mux_3sigma)),
    idd_sppd = range(len(sppd_laser_3sigma)),
    idd_bw_lambda = range(len(enablence_mux_bw_lambda)),
    idd_laser = range(no_lasers),
    idd_wpe = range(len(wpe_slope_3sigma))
  ),
  attrs = dict(description = "SPPD CLM Calibration data for Monte Carlo simulations - dated: 2024-03-09")
)

In [7]:
data.to_netcdf("../simulation_data/sppd_clm_calibration_1_mux_drive_current_tuning_calibration_mc_20240309_v0p2.nc")